# LanceDB Example

This notebook demonstrates how to use the `LanceDB` class from the `pandas-ai` library.

In [ ]:
import os
import pandas as pd
import pandasai as pai
from pandasai_lancedb import LanceDB
from pandasai_litellm.litellm import LiteLLM
from pandasai import Agent

# 1. Configure LLM and PandasAI
api_key = os.getenv("OPENAI_API_KEY", "your-api-key")
llm = LiteLLM(model="gpt-5-mini", api_key=api_key)
pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})



In [ ]:
# 2. Prepare DataFrame

sales_raw = pd.DataFrame([
    {"region": "EU", "channel": "Online", "revenue": 125000,
        "units": 420, "week": "2024-01-05"},
    {"region": "US", "channel": "Retail", "revenue": 174000,
        "units": 510, "week": "2024-01-05"},
    {"region": "APAC", "channel": "Online", "revenue": 98000,
        "units": 360, "week": "2024-01-12"},
])

sales_semantic = pai.DataFrame(
    sales_raw,
    name="global_sales",
    description="Aggregated weekly sales by region",
)



In [ ]:
# 3. Configure Vector Store
vector_store = LanceDB(persist_path="./lancedb")



In [ ]:
# 4. Initialize Agent
agent = Agent([sales_semantic], vectorstore=vector_store)



In [ ]:
# 5. Chat with the Agent (before training)
response = agent.chat("Which region leads in revenue?", output_type="string")
print(response)



In [ ]:
# 6. Train the agent
agent.train(
    queries=["How are things?"],
    codes=["SELECT region, SUM(revenue) FROM global_sales GROUP BY region Order by region asc"],
    docs=["Sales regions include EU, US and APAC"],
)



In [ ]:
# 7. Chat with the Agent (after training)
response = agent.chat("How are things?", output_type="string")
print(response)
